# Random Forest Regression

This notebook is based on [NN_SVM_RF_classification_supervised_Evaluated.ipynb](https://github.com/RichardsGroup/AGN_DataChallenge/blob/main/submissions/SER-SAG/NN_SVM_RF_classification_supervised_Evaluated.ipynb) submission to the LSST AGN Data Challenge by SER-SAG.

Authors: Djordje Savic (Postdoc), Isidora Jankov (PhD student), Iva Čvorović-Hajdinjak (PhD student)

## Data reading, selection and pre-processing

In [1]:
## commonly used modules
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys
import yaml
import seaborn as sns
import importlib

from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets 
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection as model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.metrics import f1_score
#from sklearn.metrics import plot_confusion_matrix #command is deprecated
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

pd.set_option('display.max_columns', 1000)

### Data selection
The object table follows the current version of the LSST Data Products document (LSE-163) as much as possible with measurements of included objects in the following main catalogries:
- __Astromety__ -> ra, dec, proper motion and parallax
- __Photometry__ -> point and extended source photometry, in both AB magnitdues and fluxes (nJy)
- __Color__ -> Computed using the fluxes
- __Morphology__ -> 1 for extended and 0 for point-like
- __Light Curve Features__ -> Extrated on the SDSS light curves if matched
- __Redshift__ -> Both spectroscopic and photometric, wherever available

The data used here is from the 's82ObjectTable.parquet' file available online on [Zenodo](https://zenodo.org/records/6862159).

In [2]:
sample1 = pd.read_csv('select_attributes1.csv', index_col=0)

In [3]:
sample1

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
271386,2.047289,1.542644,1.517730,0.691103,0.180547,Star,0.000000
271388,2.628649,1.437541,0.672900,0.403049,NaN,Star,0.000000
271389,0.506060,-0.086831,-0.137316,-0.160921,NaN,Star,0.000000
271390,1.318985,0.514469,0.154915,0.077669,-0.012270,Star,0.000000
271391,0.964024,0.301733,0.076412,0.028065,-0.041763,Star,0.000000
...,...,...,...,...,...,...,...
1468017,0.451338,0.154067,0.368441,0.338225,NaN,Qso,0.442569
1468018,0.065456,0.106526,-0.103106,-0.005643,NaN,Qso,1.110641
1468019,0.156834,0.123606,0.331176,0.023310,NaN,Qso,1.775506


In [4]:
# Changing the class value to numerical for using sparse categorical cross entropy loss from tensorflow
# Also moving Agn and highZQso to Qso label
sample1_good = sample1.replace({'class': {'Star': 0, 'Gal': 1, 'Qso': 2, 'Agn': 2, 'highZQso': 2}})
sample1_good

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
271386,2.047289,1.542644,1.517730,0.691103,0.180547,0.0,0.000000
271388,2.628649,1.437541,0.672900,0.403049,NaN,0.0,0.000000
271389,0.506060,-0.086831,-0.137316,-0.160921,NaN,0.0,0.000000
271390,1.318985,0.514469,0.154915,0.077669,-0.012270,0.0,0.000000
271391,0.964024,0.301733,0.076412,0.028065,-0.041763,0.0,0.000000
...,...,...,...,...,...,...,...
1468017,0.451338,0.154067,0.368441,0.338225,NaN,2.0,0.442569
1468018,0.065456,0.106526,-0.103106,-0.005643,NaN,2.0,1.110641
1468019,0.156834,0.123606,0.331176,0.023310,NaN,2.0,1.775506


In [5]:
# dropping nan values
sample1_good = sample1_good.dropna()

display(sample1_good.columns, sample1_good.describe())

Index(['stdColor_0', 'stdColor_1', 'stdColor_2', 'stdColor_3', 'stdColor_4',
       'class', 'z'],
      dtype='object')

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
count,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000
mean,1.192455,0.796617,0.458105,0.246363,0.137506,0.969585,0.699422
std,1.097236,0.577364,0.350144,0.179869,0.181409,0.717093,0.706082
min,-3.456974,-4.951052,-8.632828,-2.787663,-3.639812,0.000000,-0.006079
25%,0.383977,0.338318,0.165244,0.097926,0.010907,0.000000,0.000000
50%,0.989902,0.611921,0.444655,0.257902,0.111104,1.000000,0.602561
75%,1.892795,1.360131,0.669739,0.372838,0.225287,1.000000,0.925258
max,16.439950,8.568064,2.936810,4.250202,2.608742,2.000000,7.011245


### Data preprocessing 

In [6]:
# choose the records with class = 1 (galaxies)
only_qso = sample1_good[sample1_good['class']==2.0]
only_qso

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
697454,0.662237,1.104010,0.762233,0.219076,0.281439,2.0,0.536461
698195,1.024983,0.654579,0.254540,0.241489,0.111748,2.0,0.113405
698558,-0.088183,0.757585,0.567576,0.202108,0.258708,2.0,0.557635
698575,1.333504,1.175553,0.483101,0.384510,0.291141,2.0,0.190141
698681,1.025274,0.972344,0.268497,0.465105,0.021840,2.0,0.377762
...,...,...,...,...,...,...,...
1467479,0.480263,0.647133,0.185295,0.087828,0.197606,2.0,1.353339
1467481,0.404208,0.272697,0.055519,0.148398,-0.062617,2.0,0.903731
1467482,1.717676,-0.179669,0.315848,0.627024,0.443895,2.0,0.925798


In [7]:
only_qso.to_csv('only_qso_good.csv')

In [8]:
only_qso = pd.read_csv('only_qso_good.csv')

In [9]:
print(len(only_qso))

60856


In [10]:
#half the sample to reduce computational time
only_qso = only_qso[:int((len(only_qso)-1)/4)]

In [11]:
print(len(only_qso))

15213


In [12]:
#splitting into X and y values

X = only_qso
# removing the z attribute from X data array
X = X.drop(['z'], axis=1)

# storing labels for later
y = only_qso['z']

display(X.shape, y.shape)
display(X.describe()) #no z property

(15213, 7)

(15213,)

,objectId,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class
count,1.521300e+04,15213.000000,15213.000000,15213.000000,15213.000000,15213.000000,15213.0
mean,1.370641e+06,0.517402,0.284194,0.168841,0.135233,0.069620,2.0
std,1.159060e+05,0.715632,0.304400,0.178988,0.157613,0.162580,0.0
min,6.974540e+05,-2.164861,-0.670916,-0.389593,-0.669849,-0.611229,2.0
25%,1.389461e+06,0.098085,0.095501,0.036270,0.014658,-0.036686,2.0
50%,1.394639e+06,0.395802,0.227825,0.150691,0.123048,0.051670,2.0
75%,1.400418e+06,0.779555,0.370612,0.273548,0.244158,0.148146,2.0
max,1.405383e+06,10.609983,2.925405,1.800148,0.992748,1.262812,2.0


In [13]:
## Splitting data to train, test followed by standardization
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.5, random_state = 1)

## standardization using StandardScaler applied to X_train (yields mean and sigma for X_train)
## and then standardizing X_train and X_test with mean and sigma obtained from X_train

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

display(X_train.shape, X_test.shape)

(7606, 7)

(7607, 7)

## Random forest regression
Create a random forest classifier.

In [14]:
clf=RandomForestRegressor(n_estimators=2001, n_jobs=70, criterion="squared_error")

Source: [scikit-learn](https://scikit-learn.org/dev/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [15]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=2001, n_jobs=70)

In [16]:
y_pred_rf=clf.predict(X_test)

In [17]:
display(f"MSE: {mean_squared_error(y_test, y_pred_rf)}")
display(f"MAE: {mean_absolute_error(y_test, y_pred_rf)}")
display(f"R2 score: {r2_score(y_test, y_pred_rf)}")

'MSE: 0.2637402204420231'

'MAE: 0.3316828132846307'

'R2 score: 0.5043859151185783'

R^2 score (coefficient of determination) represents how well predictions of a regression model fit the actual data.